In [ ]:
import qiskit
import qiskit.qasm3
import openqasm3

In [ ]:
qc = qiskit.QuantumCircuit(2)
qc.h(0)
qc.barrier()
qc.cx(0, 1)
qc.barrier()
qc.measure_all()

In [ ]:
with open('test.qasm', 'w') as f:
    qiskit.qasm3.dump(qc, f)

In [ ]:
#with open ('test.qasm', 'r') as f:
with open ('HHL_circuit_Qiskit_2x2.qasm3', 'r') as f:
    qasm = f.read()

print(qasm)

In [ ]:
qasm_ast_program = openqasm3.ast.Program(qasm)
#openqasm3.parse(qasm)

In [ ]:
qasm_ast_program.span

In [ ]:
qasm_parsed_file= openqasm3.parse(qasm)

In [ ]:
qasm_parsed_file.span

In [ ]:
for s in qasm_parsed_file.statements:
    print(s)

In [ ]:
qasm_parsed_file.version

In [ ]:
state = openqasm3.visitor.QASMVisitor().visit(node=qasm_parsed_file)

In [ ]:
state

In [ ]:
qasm_parsed_file.statements

In [ ]:
for statement in qasm_parsed_file.statements:
    print(statement)

In [ ]:
for statement in qasm_parsed_file.statements:
    if isinstance(statement, openqasm3.ast.Include):
        print('Included filenames:', statement.filename)
    elif isinstance(statement, openqasm3.ast.ClassicalDeclaration):
        if isinstance(statement.type, openqasm3.ast.BitType):
            cbit_type = 'bit'
        else:
            raise ValueError('Unknown type', 'statement.type:', statement.type)
        cbit_name = statement.identifier.name
        cbit_length = statement.type.size.value
        print('Classical bit name:', cbit_name)
        print('Classical bit type:', cbit_type)
        print('Classical bit length:', cbit_length)
    elif isinstance(statement, openqasm3.ast.QubitDeclaration):
        qbit_name = statement.qubit.name
        qbit_length = statement.size.value
        print('Quantum bit name:', qbit_name)
        print('Quantum bit length:', qbit_length)
    elif isinstance(statement, openqasm3.ast.QuantumGate):
        qgate_name = statement.name.name
        qgate_qbit_names = [statement.qubits[i].name.name for i in range(len(statement.qubits))]
        qgate_qbit_indices = [statement.qubits[i].indices[0][0].value for i in range(len(statement.qubits))] # why is there doubly nested list in indices?
        print('Quantum gate name:', qgate_name)
        print('Quantum gate qubits names:', qgate_qbit_names)
        print('Quantum gate qubits indices:', qgate_qbit_indices)
    elif isinstance(statement, openqasm3.ast.QuantumBarrier):
        #print(statement)
        qbarrier_name = 'barrier' # name is not stored in statement 
        qbarrier_qbit_names = [statement.qubits[i].name.name for i in range(len(statement.qubits))]
        qbarrier_qbit_indices = [statement.qubits[i].indices[0][0].value for i in range(len(statement.qubits))] # why is there doubly nested list in indices?
        print('Quantum gate name:', qbarrier_name)
        print('Quantum gate qubits names:', qbarrier_qbit_names)
        print('Quantum gate qubits indices:', qbarrier_qbit_indices)
    elif isinstance(statement, openqasm3.ast.QuantumMeasurementStatement):
        #print(statement)
        qmeasurement_name = 'measurement' # name is not stored in statement 
        qmeasurement_qbit_name = statement.measure.qubit.name.name
        qmeasurement_qbit_index = statement.measure.qubit.indices[0][0].value
        qmeasurement_cbit_name = statement.target.name.name
        qmeasurement_cbit_index = statement.target.indices[0][0].value
        print('Quantum measurement name:', qmeasurement_name)
        print('Quantum measurement qubit name:', qmeasurement_qbit_name)
        print('Quantum measurement qubit index:', qmeasurement_qbit_index)
        print('Quantum measurement cbit name:', qmeasurement_cbit_name)
        print('Quantum measurement cbit index:', qmeasurement_cbit_index)

######
#
# the statements are ordered in some logical timeseries manner. A gate can be interpreted as a definite border between two time steps.
#
######

In [ ]:
import QSymPy

In [ ]:
qcs = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)
qcs.add_gate('H', qubits_t=[0], step=0)
qcs.add_barrier(step=0)
qcs.add_gate('CX', qubits_c=[0], qubits_t=[1], step=1)
qcs.add_barrier(step=1)

qcs.assemble_symbolic_unitary()
qcs.unitary


In [ ]:
for e in qcs.unitary[0,0].free_symbols: break
e.name

In [ ]:
qcs.barrier_collection

In [ ]:
qcs2 = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)
qcs2.add_gate(name='U', qubits_t=[0], step=0, parameters={'theta': 0.5, 'phi': 0.5, 'lambda': 0.5})
qcs2.add_gate(name='X', qubits_t=[1], step=0)
qcs2.assemble_symbolic_unitary()
qcs2.unitary

In [ ]:
qcs2.gate_collection.collections['U'][0].matrix_alt

In [ ]:
#subs_symbolic_alternatives_in_symbolic_unitary
qcs2.unitary

In [ ]:
#qcs2.subs_symbolic_alternatives_in_symbolic_unitary({'steps': (0,0), 'names': ('U','U'), 'qubits_t': (0,1)})
qcs2.subs_symbolic_alternatives_in_symbolic_unitary()
qcs2.unitary

In [ ]:
qcs3 = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)
qcs3.add_gate(name='GP', qubits_t=[0], step=0, parameters={'gamma': 0.5})
qcs3.assemble_symbolic_unitary()
qcs3.unitary

In [ ]:
qcs3.subs_symbolic_alternatives_in_symbolic_unitary()
qcs3.unitary

In [ ]:
qcs4 = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)
qcs4.add_gate(name='P', qubits_t=[0], step=0, parameters={'lambda': 0.5})
qcs4.assemble_symbolic_unitary()
qcs4.unitary

In [ ]:
qcs4.subs_symbolic_alternatives_in_symbolic_unitary()
qcs4.unitary

In [ ]:
import numpy as np
qcs2.gate_collection.collections['U'][0].matrix_numeric


In [ ]:
np.array(qcs2.gate_collection.collections['U'][0].matrix_numeric)

In [ ]:
qcs2.gate_collection.collections['U'][0].atomics

In [ ]:
qcs2.subs_symbolic_zerosones_in_symbolic_unitary()
qcs2.unitary

In [ ]:
qcs2.create_numeric_unitary_from_symbolic()
qcs2.unitary_numeric

In [ ]:
qc_imported = QSymPy.openqasm3_to_qc('test.qasm')
#qc_imported = QSymPy.openqasm3_to_qc('HHL_circuit_Qiskit_2x2.qasm3')

In [ ]:
qc_imported.assemble_symbolic_unitary()
qc_imported.unitary

In [ ]:
qcs4 = QSymPy.QuantumCircuit(num_qubits=3, num_clbits=2)
qcs4.add_gate(name='CX', qubits_t=[0], qubits_c=[1], step=0)
qcs4.assemble_symbolic_unitary()
qcs4.unitary

In [ ]:
qcs4.gate_collection.collections['CX'][0].matrix

In [ ]:
qcs4.gate_collection.collections['CX'][0].atomics

In [ ]:
qcs4.assemble_symbolic_unitary()
qcs4.unitary

In [ ]:
qcs4.subs_symbolic_zerosones_in_symbolic_unitary()
qcs4.unitary

In [ ]:
qcs4.unitary.atoms()

In [ ]:
qcs4.unitary.subs({'CX_X_qt0_qc1_s0_p01': -1})

In [ ]:
qcs4.create_numeric_unitary_from_symbolic()
qcs4.unitary_numeric

In [ ]:
qcs5 = QSymPy.QuantumCircuit(num_qubits=3, num_clbits=2)
qcs5.add_gate(name='CCX', qubits_t=[0], qubits_c=[1, 2], step=0)
qcs5.assemble_symbolic_unitary()
qcs5.unitary

In [ ]:
qcs5.create_numeric_unitary_from_symbolic()
qcs5.unitary_numeric

In [ ]:
qcs5.gate_collection.collections['CCX'][0].matrix

In [ ]:
qcs6 = QSymPy.QuantumCircuit(num_qubits=4, num_clbits=2)
qcs6.add_gate(name='CCXX', qubits_t=[0,1], qubits_c=[2, 3], step=0)
qcs6.assemble_symbolic_unitary()
qcs6.unitary

In [ ]:
qcs6.create_numeric_unitary_from_symbolic()
qcs6.unitary_numeric

In [ ]:
qcs7 = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)
qcs7.add_gate(name='CU', qubits_t=[0], qubits_c=[1], step=0, parameters={'theta': 0.5, 'phi': 0.5, 'lambda': 0.5})
qcs7.assemble_symbolic_unitary()
qcs7.unitary

In [ ]:
qcs7.subs_symbolic_alternatives_in_symbolic_unitary()
qcs7.unitary

In [ ]:
qcs8 = QSymPy.QuantumCircuit(num_qubits=3, num_clbits=2)
qcs8.add_gate(name='CUU', qubits_t=[0,1], qubits_c=[2], step=0, parameters=[{'theta': 0.5, 'phi': 0.5, 'lambda': 0.5}, {'theta': 0.5, 'phi': 0.5, 'lambda': 0.5}])
qcs8.assemble_symbolic_unitary()
qcs8.unitary

In [ ]:
qcs8.subs_symbolic_alternatives_in_symbolic_unitary()
qcs8.unitary

In [ ]:
qcs8.subs_symbolic_zerosones_in_symbolic_unitary(zeros=True, ones=True)
qcs8.unitary